# Training an Acoustic Model with subword tokenization

In [ ]:
# Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Grab the config we'll use in this example
!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/citrinet/config_bpe.yaml

"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""
# exit()

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.4-1ubuntu0.1).
libsndfile1 is already the newest version (1.0.28-7ubuntu0.1).
sox is already the newest version (14.4.2+git20190427-2).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-3hdq52c5/nemo-toolkit_041a7e8147f4492ea4407389b985771b
  Running command git clone -q https://github.com/NVIDIA/NeMo.git /tmp/pip-install-3hdq52c5/nemo-toolkit_041a7e8147f4492ea4407389b985771b


Let's begin constructing an ASR model that will use the subword tokenizer for its dataset pre-processing and post-processing steps.

We will use a Citrinet model to demonstrate the usage of subword tokenization models for training and inference. Citrinet is a [QuartzNet-like architecture](https://arxiv.org/abs/1910.10261), but it uses subword-tokenization along with 8x subsampling and [Squeeze-and-Excitation](https://arxiv.org/abs/1709.01507) to achieve strong accuracy in transcriptions while still using non-autoregressive decoding for efficient inference.

We'll be using the **Neural Modules (NeMo) toolkit** for this part, so if you haven't already, you should download and install NeMo and its dependencies. To do so, just follow the directions on the [GitHub page](https://github.com/NVIDIA/NeMo), or in the [documentation](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/).

NeMo let us easily hook together the components (modules) of our model, such as the data layer, intermediate layers, and various losses, without worrying too much about implementation details of individual parts or connections between modules. NeMo also comes with complete models which only require your data and hyperparameters for training.

In [ ]:
# NeMo's "core" package
import nemo
print(nemo.__version__)
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr

## Cross-Language Transfer Learning

Transfer learning is an important machine learning technique that uses a model’s knowledge of one task to perform better on another. Fine-tuning is one of the techniques to perform transfer learning. It is an essential part of the recipe for many state-of-the-art results where a base model is first pretrained on a task with abundant training data and then fine-tuned on different tasks of interest where the training data is less abundant or even scarce.

In ASR you might want to do fine-tuning in multiple scenarios, for example, when you want to improve your model's performance on a particular domain (medical, financial, etc.) or accented speech. You can even transfer learn from one language to another! Check out [this paper](https://arxiv.org/abs/2005.04290) for examples.

Transfer learning with NeMo is simple.

-----
First, let's create another tokenizer - perhaps using a larger vocabulary size than the small tokenizer we created earlier. Also we swap out `sentencepiece` for `BERT Word Piece` tokenizer.

In [ ]:
import nemo.collections.asr as nemo_asr
asr_model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_conformer_ctc_large")

In [ ]:
# Check what kind of vocabulary/alphabet the model has right now
print(asr_model.decoder.vocabulary)

In [ ]:
len(asr_model.decoder.vocabulary)

Now let's update the vocabulary in this model

In [ ]:
# Lets change the tokenizer vocabulary by passing the path to the new directory,
# and also change the type
asr_model.change_vocabulary(
    new_tokenizer_dir="../data_preparation/data/processed/tokenizer/tokenizer_spe_bpe_v1024/",
    new_tokenizer_type="bpe"
)

After this, our decoder has completely changed, but our encoder (where most of the weights are) remained intact. Let's fine tune-this model for 20 epochs on AN4 dataset. We will also use the smaller learning rate from ``new_opt` (see the "After Training" section)`.

**Note**: For this demonstration, we will also freeze the encoder to speed up finetuning (since both tokenizers are built on the same train set), but in general it should not be done for proper training on a new language (or on a different corpus than the original train corpus).

In [ ]:
vars(asr_model)

In [ ]:
## Grab the config we'll use in this example
BRANCH='main'
!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/conformer/conformer_ctc_bpe.yaml

In [ ]:
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

with initialize(config_path="./configs/"):
    cfg = compose(config_name="conformer_ctc_bpe.yaml")
    print(cfg)
    

In [ ]:
print(OmegaConf.to_yaml(cfg))

In [ ]:
cfg['model']['train_ds']

In [ ]:
sample_rate = cfg.model.sample_rate
print(f"The sample_rate = {sample_rate}")

ds_sample_rate = cfg.model.train_ds.sample_rate
print(f"The ds_sample_rate = {ds_sample_rate}")


In [ ]:
params = cfg

import copy
new_opt = copy.deepcopy(params.model.optim)
new_opt.lr = 0.1

In [ ]:
# Update paths to dataset
params.model.train_ds.manifest_filepath = '../data_preparation/data/processed/train_manifest_merged.json'
params.model.train_ds.sample_rate = 16000
params.model.train_ds.batch_size = 1

params.model.validation_ds.manifest_filepath = ['../data_preparation/data/processed/test_manifest_merged.json', '../data_preparation/data/processed/dev_manifest_merged.json']
params.model.validation_ds.batch_size = 1

In [ ]:
params['model']['train_ds']

In [ ]:
params.model.encoder.d_model

In [ ]:
# Use the smaller learning rate we set before
asr_model.setup_optimization(optim_config=new_opt)

In [ ]:
asr_model._cfg.optim.sched.d_model = params.model.encoder.d_model
asr_model._cfg.trainer.log_every_n_steps = 1000

In [ ]:
params.model.optim.sched.d_model = params.model.encoder.d_model
print(params.model.optim.sched.d_model)

# Point to the data we'll use for fine-tuning as the training set
asr_model.setup_training_data(train_data_config=params.model.train_ds)

# Point to the new validation data for fine-tuning
asr_model.setup_validation_data(val_data_config=params['model']['validation_ds'])

# Freeze the encoder layers (should not be done for finetuning, only done for demo)
asr_model.encoder.freeze()

# Load the TensorBoard notebook extension
if COLAB_ENV:
  %load_ext tensorboard
  %tensorboard --logdir lightning_logs/
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

In [ ]:
!ln -s ../data_preparation/data data

In [ ]:
# And now we can create a PyTorch Lightning trainer and call `fit` again.
import pytorch_lightning as pl

trainer = pl.Trainer(devices=1, accelerator='gpu', max_epochs=20)
trainer.fit(asr_model)

So we get fast convergence even though the decoder vocabulary is double the size and we freeze the encoder.

### Fast Training

Last but not least, we could simply speed up training our model! If you have the resources, you can speed up training by splitting the workload across multiple GPUs. Otherwise (or in addition), there's always mixed precision training, which allows you to increase your batch size.

You can use [PyTorch Lightning's Trainer object](https://pytorch-lightning.readthedocs.io/en/latest/common/trainer.html?highlight=Trainer) to handle mixed-precision and distributed training for you. Below are some examples of flags you would pass to the `Trainer` to use these features:

```python
# Mixed precision:
trainer = pl.Trainer(amp_level='O1', precision=16)

# Trainer with a distributed backend:
trainer = pl.Trainer(devices=2, num_nodes=2, accelerator='gpu', strategy='dp')

# Of course, you can combine these flags as well.
```

Finally, have a look at [example scripts in NeMo repository](https://github.com/NVIDIA/NeMo/blob/stable/examples/asr/asr_ctc/speech_to_text_ctc_bpe.py) which can handle mixed precision and distributed training using command-line arguments.